# Raw Data Downloading

### The goal of this notebook is to extract raw data from web pages, no modifications are performed on this stage

## Data, the link of the source, and web scraping description

#### 1. Latest Data Science Job Salaries 2020 - 2025 (Main Dataset)
__Link__: https://www.kaggle.com/datasets/saurabhbadole/latest-data-science-job-salaries-2024

__How data was collected__: Directly downloaded from the web page as a zip file with CSV and JSON formats available. Therefore, in this notebook no operations were done with this source.

#### 2. Unemployment Rate by Country and Year (1990-2024)
__Link__: https://w3.unece.org/PXWeb2015/pxweb/en/STAT/STAT__20-ME__3-MELF/40_en_MEUnRateY_r.px/

__How data was collected__: Since the website is an interface that sends a query to the database, I wrote a similar POST request to get the table with all countries and years available. Method: documented statistical API (PXWeb)

#### 3. Inflation Rate by Country (2019-2023)
__Link__: https://www.macrotrends.net/global-metrics/countries/ranking/inflation-rate-cpi

__How data was collected__: Having HTML code with the table, I used __requests__ and __Beautiful Soup__ libraries for data parsing.

#### 4. The Happy Planet Index 2024 (2006-2021)
__Link__: https://happyplanetindex.org/countries/

__How data was collected__: Directly downloaded .xlsx file with data. Since information by years was updating on the same table after choosing the year in the list, I manually copied tables in the separate file, placing data on separate sheets. Then joined tables (by country name) using Pandas.


#### 5. Corruption Perceptions Index (1995-2025)
__Link__: https://www.transparency.org/en/cpi/2024

__How data was collected__: Reverse-engineering client-side API. The process is similar to source 2, but instead of interacting with a documented statistical API, the endpoint was discovered through browser network inspection.

#### 6. Индекс счастья(2014-2024)
__Link__: https://statbase.ru/datasets/indexes-and-ratings/happiness-index/

__How data was collected__: via automated HTML parsing. Since the website supports year filtering via URL parameters (?syear=YYYY), we iterate over years 2014–2024 and parse structured div-based table rows into JSON format using BeautifulSoup.

In [281]:
## Import of all necessary libraries

import pandas as pd # to see parsed data in convenient format as DataFrames 
import requests # 
import json
from bs4 import BeautifulSoup # for source 3

## 2. Unemployment Rate by Country and Year (1990-2024)

### Step 0: understand the structure of API requests to the database

Using DevTools (Network), I filtered types of requests (Fetch, XHR) that return the table with data.

Inside them I obtained PXWeb API in the form: ``[{"dbid":"STAT","text":"STAT"}]``

Moreover, it follows tree structure with structured data.


After sending following request: https://w3.unece.org/PXWeb2015/api/v1/en/STAT/20-ME/3-MELF/40_en_MEUnRateY_r.px, I received JSON with table structure. 
### Step 1: Request
Using obtained information (schema), I composed POST request to the same endpoint (variable url_2 below)

In [179]:
url_2 = "https://w3.unece.org/PXWeb2015/api/v1/en/STAT/20-ME/3-MELF/40_en_MEUnRateY_r.px"

In [181]:
# PXWeb waits for the request in this form:
payload = {
    "query": [
        {
            "code": "Country",
            "selection": {
                "filter": "all",
                "values": ["*"]
            }
        },
        {
            "code": "Year",
            "selection": {
                "filter": "all",
                "values": ["*"]
            }
        }
    ],
    "response": {
        "format": "json"
    }
}

response_2 = requests.post(url_2, json=payload)

In [183]:
# Saving raw data in JSON file
data_2 = response_2.json()

with open("unemployment_raw.json", "w", encoding="utf-8") as f:
    json.dump(data_2, f, ensure_ascii=False, indent=2)

In [185]:
print(data_2)

{'columns': [{'code': 'Country', 'text': 'Country', 'type': 'd'}, {'code': 'Year', 'text': 'Year', 'type': 'd'}, {'code': 'Unemployment Rate', 'text': 'Unemployment Rate', 'type': 'c'}], 'comments': [], 'data': [{'key': ['008', '0'], 'values': ['9.5']}, {'key': ['008', '1'], 'values': ['9.1']}, {'key': ['008', '2'], 'values': ['26.5']}, {'key': ['008', '3'], 'values': ['22.3']}, {'key': ['008', '4'], 'values': ['18.4']}, {'key': ['008', '5'], 'values': ['12.9']}, {'key': ['008', '6'], 'values': ['12.3']}, {'key': ['008', '7'], 'values': ['14.9']}, {'key': ['008', '8'], 'values': ['17.7']}, {'key': ['008', '9'], 'values': ['18.4']}, {'key': ['008', '10'], 'values': ['16.8']}, {'key': ['008', '11'], 'values': ['16.4']}, {'key': ['008', '12'], 'values': ['4.6']}, {'key': ['008', '13'], 'values': ['15.2']}, {'key': ['008', '14'], 'values': ['14.8']}, {'key': ['008', '15'], 'values': ['14.3']}, {'key': ['008', '16'], 'values': ['13.9']}, {'key': ['008', '17'], 'values': ['13.4']}, {'key': [

## 3. Inflation Rate by Country (2019-2023)

In [285]:
# Sending request
url_3 = "https://www.macrotrends.net/global-metrics/countries/ranking/inflation-rate-cpi"

headers_3 = {"User-Agent": "Mozilla/5.0"}
response_3 = requests.get(url, headers=headers_3)

In [287]:
# Parsing a table from the page
soup = BeautifulSoup(response_3.text, "html.parser")

table_3 = soup.find("table")

In [270]:
# Extracting headers and rows of the table
headers_3 = []
rows_3 = []

for th in table_3.find_all("th"):
    headers_3.append(th.text.strip())

for tr in table_3.find_all("tr")[1:]:  # skipping headers
    cells = tr.find_all("td")
    if cells:
        row = [cell.text.strip() for cell in cells]
        rows_3.append(row)

In [289]:
inflation_df = pd.DataFrame(rows_3, columns=headers_3)

In [291]:
# Storing raw file in JSON format
inflation_df.to_json("raw_data\inflation_data.json", orient="records", indent=4)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Honor\AppData\Local\Temp\ipykernel_22836\3118366961.py:2: SyntaxWarning: invalid escape sequence '\i'
  inflation_df.to_json("raw_data\inflation_data.json", orient="records", indent=4)


## 4. The Happy Planet Index 2024 (2006-2021)

In [197]:
file_4 = "happy_planet.xlsx"
years = range(2006, 2022)

xls = pd.ExcelFile(file_4)
print(xls.sheet_names) # looking at the list of sheets in the file

['raw_2006', 'raw_2007', 'raw_2008', 'raw_2009', 'raw_2010', 'raw_2011', 'raw_2012', 'raw_2013', 'raw_2014', 'raw_2015', 'raw_2016', 'raw_2017', 'raw_2018', 'raw_2019', 'raw_2020', 'raw_2021']


In [199]:
df_4 = None

In [201]:
for year in years:
    sheet_name = f"raw_{year}"
    
    df = pd.read_excel(file_4, sheet_name=sheet_name)
    
    # Choosing necessary columns
    df_year = df[[
        "Country",
        "ISO",
        "Continent",
        "HPI rank",
        "Population (thousands)",
        "Life Expectancy (years)",
        "HPI"
    ]]
    
    # Renaming columns
    df_year = df_year.rename(columns={
        "HPI rank": f"HPI_rank_{year}",
        "Population (thousands)": f"Population_{year}",
        "Life Expectancy (years)": f"LifeExp_{year}",
        "HPI": f"HPI_{year}"
    })
    
    if df_4 is None:
        df_4 = df_year
    else:
        df_4 = df_4.merge( # Joining tables by different years in one
            df_year,
            on=["Country", "ISO", "Continent"],
            how="outer"
        )

In [202]:
print(df_4.shape) # ensuring the size of final df

(149, 67)


In [205]:
df_4.head()

,Country,ISO,Continent,HPI_rank_2006,Population_2006,LifeExp_2006,HPI_2006,HPI_rank_2007,Population_2007,LifeExp_2007,...,LifeExp_2019,HPI_2019,HPI_rank_2020,Population_2020,LifeExp_2020,HPI_2020,HPI_rank_2021,Population_2021,LifeExp_2021,HPI_2021
0,Afghanistan,AFG,6,NaN,25442.944,58.684,NaN,NaN,25903.301,59.111,...,63.565,16.936078,146.0,38972.230,62.575,16.455519,144.0,40099.462,61.982,16.210671
1,Albania,ALB,7,NaN,3003.387,76.816,NaN,17.0,2976.084,77.549,...,79.282,46.633530,26.0,2866.849,76.989,48.423285,33.0,2854.710,76.463,46.333546
2,Algeria,DZA,4,NaN,33435.080,72.334,NaN,NaN,33983.827,72.602,...,76.474,43.535362,30.0,43451.666,74.453,47.547291,29.0,44177.969,76.377,47.446084
3,Argentina,ARG,1,12.0,39476.851,75.433,46.402602,23.0,39876.111,75.006,...,77.284,46.700021,40.0,45036.032,75.892,44.896712,41.0,45276.780,75.390,43.792331
4,Armenia,ARM,7,35.0,3026.486,71.987,37.629462,39.0,3004.393,72.327,...,75.439,47.785129,44.0,2805.608,72.173,44.389059,45.0,2790.974,72.043,43.001950


In [207]:
df_4.to_json("raw_data\happiness_data.json", orient="records", indent=4)

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Honor\AppData\Local\Temp\ipykernel_22836\2416058097.py:1: SyntaxWarning: invalid escape sequence '\h'
  df_4.to_json("raw_data\happiness_data.json", orient="records", indent=4)


## 5. Corruption Perceptions Index (1995-2025)

### Step 0: similarly to source 2, I started with understanding the structure of the request
Using DevTools, I filtered XHR/Fetch requests while reloading the CPI page. Then found that the website dynamically retrieves data using GET request to the endpoint (variable url_5). The response returns structured JSON containing country-level CPI scores across multiple years.

### Step 1: direct API Request

In [210]:
url_5 = "https://www.transparency.org/en/api/latest/cpi"

response_5 = requests.get(url_5)
data_5 = response_5.json()

In [212]:
print(len(data_5))  # ensure that there are enough rows parsed

2494


In [214]:
df_5 = pd.DataFrame(data_5)
df_5.head(5) # just glancing at what we have retrieved

,country,iso3,region,year,score,rank,sources,standardError,lowerCi,upperCi
0,Afghanistan,AFG,AP,2012,8,174,3,3.30,2,13
1,Afghanistan,AFG,AP,2013,8,175,3,3.30,3,13
2,Afghanistan,AFG,AP,2014,12,172,4,1.29,10,14
3,Afghanistan,AFG,AP,2015,11,166,4,3.49,5,17
4,Afghanistan,AFG,AP,2016,15,169,5,1.74,12,17


In [216]:
# Saving raw data in JSON format
with open("cpi_raw.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## 6. Индекс счастья (2014-2024)

In [218]:
data_6 = [] # storing parsed info

In [220]:
for year in range(2014, 2025): # iterating over 10 years
    url_6 = f"https://statbase.ru/datasets/indexes-and-ratings/happiness-index/?syear={year}"
    response_6 = requests.get(url)
    soup = BeautifulSoup(response_6.text, "html.parser")

    rows = soup.find_all("div", class_="it_row cnshort")

    for row in rows:
        cells = row.find_all("div", class_="it_cell")
        if len(cells) >= 3:
            country = cells[0].get_text(strip=True)
            score = cells[2].get_text(strip=True).replace(",", ".")
            rank = cells[3].get_text(strip=True) if len(cells) > 3 else None

            data_6.append({
                "year": year,
                "country": country,
                "score": float(score) if score else None,
                "rank": int(rank) if rank and rank.isdigit() else None
            })

In [222]:
with open("happiness_raw.json", "w", encoding="utf-8") as f: # Saving raw file
    json.dump(data_6, f, ensure_ascii=False, indent=4)

In [224]:
print(data_6[0:10]) # just glancing at the data

[{'year': 2014, 'country': 'Финляндия', 'score': 7.736, 'rank': 1}, {'year': 2014, 'country': 'Дания', 'score': 7.521, 'rank': 2}, {'year': 2014, 'country': 'Исландия', 'score': 7.515, 'rank': 3}, {'year': 2014, 'country': 'Швеция', 'score': 7.345, 'rank': 4}, {'year': 2014, 'country': 'Нидерланды', 'score': 7.306, 'rank': 5}, {'year': 2014, 'country': 'Коста-Рика', 'score': 7.274, 'rank': 6}, {'year': 2014, 'country': 'Норвегия', 'score': 7.262, 'rank': 7}, {'year': 2014, 'country': 'Израиль', 'score': 7.234, 'rank': 8}, {'year': 2014, 'country': 'Люксембург', 'score': 7.122, 'rank': 9}, {'year': 2014, 'country': 'Мексика', 'score': 6.979, 'rank': 10}]


## The next step is to analyze obtained data and modify a bit its format. Goal: store it in the Database in the convenient form for further manipulations on it